# Test backend (pose graph)

In [38]:
import numpy as np
from graphslam.load import load_g2o_se3
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.pose.se3 import PoseSE3
from graphslam.edge.edge_odometry import EdgeOdometry
import plotly.graph_objects as go

from lgchimera.geom_util import R_to_quat
from lgchimera.pose_graph import PoseGraph

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Perfect measurements

In [22]:
# Node positions
nodes = []
node_positions = np.array([[0, 0, 0],
                           [1, 0, 0],
                           [2, 0, 0],
                           [3, 0, 0],
                           [4, 0, 0],
                           [5, 0, 0]])

N = len(node_positions)
for i, pos in enumerate(node_positions):
    p = PoseSE3(pos, R_to_quat(np.eye(3)))
    nodes.append(Vertex(i, p))

In [23]:
# Lidar odometry edges
# R = I for all edges
lidar_edges = []

lidar_ts = np.diff(node_positions, axis=0)
lidar_R = np.eye(3)
for i, t in enumerate(lidar_ts):
    estimate = PoseSE3(t, R_to_quat(np.eye(3)))
    e = EdgeOdometry([i, i+1], np.eye(6), estimate)
    lidar_edges.append(e)

In [24]:
# GPS position measurements
GPS_positions = np.array([[0, 0, 0],
                          [1, 0, 0],
                          [2, 0, 0],
                          [3, 0, 0],
                          [4, 0, 0],
                          [5, 0, 0]])

gps_nodes = []
gps_edges = []
for i, pos in enumerate(GPS_positions):
    # Create node at GPS position
    p = PoseSE3(pos, R_to_quat(np.eye(3)))
    gps_nodes.append(Vertex(i+N, p, fixed=True))
    # Connect with (R=I,t=0) edge
    estimate = PoseSE3(np.zeros(3), R_to_quat(np.eye(3)))
    e = EdgeOdometry([i, i+N], np.eye(6), estimate)
    gps_edges.append(e)

In [39]:
graph = PoseGraph(lidar_edges + gps_edges, nodes + gps_nodes)

In [40]:
graph.optimize()

In [48]:
points = np.array([[0,0,0],[0,0,1],[1,2,3],[1,-1,2]])
factors = go.Scatter3d(x=[points[:,0]], y=[points[:,1]], z=[points[:,2]], showlegend=False, 
            marker=dict(size=10, color='orange', symbol='square'))

In [53]:
fig = go.Figure(data=graph.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

## "Spoofed" GPS measurements

In [59]:
# GPS position measurements
GPS_positions = np.array([[0, 0, 0],
                          [1, 0, 0],
                          [2, 10, 0],  # Spoofed
                          [3, 0, 0],
                          [4, 0, 0],
                          [5, 0, 0]])

gps_nodes = []
gps_edges = []
for i, pos in enumerate(GPS_positions):
    # Create node at GPS position
    p = PoseSE3(pos, R_to_quat(np.eye(3)))
    gps_nodes.append(Vertex(i+N, p, fixed=True))
    # Connect with (R=I,t=0) edge
    estimate = PoseSE3(np.zeros(3), R_to_quat(np.eye(3)))
    e = EdgeOdometry([i, i+N], np.eye(6), estimate)
    gps_edges.append(e)

In [60]:
graph = PoseGraph(lidar_edges + gps_edges, nodes + gps_nodes)

In [62]:
graph.optimize()

In [63]:
fig = go.Figure(data=graph.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [66]:
for e in graph.graph._edges:
    print(e.calc_chi2())

3.8515301462564424
35.835544514843804
21.16199579354339
2.341072330282115
0.02850967434316757
0.0
3.8515301462564464
14.857933814730542
3.921760865262654
0.04299984588402305
0.001439633366841659


## Graph formation

In [69]:
g = PoseGraph()

# Initial node positions
node_positions = np.array([[0, 0, 0],
                           [1, 0, 0],
                           [2, 0, 0],
                           [3, 0, 0],
                           [4, 0, 0],
                           [5, 0, 0]])

# GPS position measurements
GPS_positions = np.array([[0, 0, 0],
                          [1, 0, 0],
                          [2, 0, 0],
                          [3, 0, 0],
                          [4, 0, 0],
                          [5, 0, 0]])

In [71]:
# Nodes
for i, pos in enumerate(node_positions):
    g.add_node(i, (np.eye(3), pos))

# Lidar odometry edges
# R = I for all edges
lidar_ts = np.diff(node_positions, axis=0)
lidar_R = np.eye(3)
for i, t in enumerate(lidar_ts):
    g.add_edge([i, i+1], (lidar_R, t))

# GPS factors
for i, pos in enumerate(GPS_positions):
    g.add_factor(i, (np.eye(3), pos))

In [73]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

TypeError: 'NoneType' object is not subscriptable

In [76]:
a = np.array([0,0,0])
np.array([0,0,a])

C:\Users\adamd\AppData\Local\Temp\ipykernel_12448\3750645195.py:2: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



array([0, 0, array([0, 0, 0])], dtype=object)

In [77]:
from scipy.spatial.transform import Rotation as R

def euler2quat(x, y, z):
    """Euler to quaternion conversion
    
    """
    r = R.from_euler('XYZ', [x, y, z])
    return r.as_quat()

In [78]:
euler2quat(4.49e-4, -1.58e-4, -1.908e-4)

array([ 2.24507533e-04, -7.89785803e-05, -9.54177327e-05,  9.99999967e-01])